# Copyright Netherlands eScience Center <br>
**Function     : Lorens-84 model ensemble generator** <br>
**Author       : Yang Liu** <br>
**First Built  : 2020.04.20** <br>
**Last Update  : 2020.04.20** <br>
**Library      : Pytorth, Numpy, NetCDF4, os, iris, cartopy, dlacs, matplotlib**<br>
Description     : This notebook serves to generated an ensemble of Lorens 84 models by perturbating initial conditions / model parameters / external forcing terms. <br>

Three different types of uncertainties will vbe investigated here:<br>
- uncertainty of model outcome attributed to perturbation in initial conditions
- model uncertainty
- uncertainty related to extra/bias/trend terms

The Lorens-84 model is described by Edward Lorens in his 1984 paper:<br>
Lorenz, E. N. (1984). Irregularity: A fundamental property of the atmosphere. Tellus A, 36(2), 98-110.<br>

Return Values   : Time series, figures and netCDF files.<br>

In [1]:
%matplotlib inline

import sys
import warnings
import numbers

# for data loading
import os
from netCDF4 import Dataset
# for pre-processing and machine learning
import numpy as np
import sklearn
#import scipy
import torch
import torch.nn.functional

#sys.path.append(os.path.join('C:','Users','nosta','ML4Climate','Scripts','DLACs'))
#sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DLACs")
sys.path.append("../")
import dlacs
import dlacs.BayesConvLSTM
import dlacs.preprocess
import dlacs.function
import dlacs.saveNetCDF

# for visualization
import dlacs.visual
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from mpl_toolkits.mplot3d import Axes3D
import iris # also helps with regriding
import cartopy
import cartopy.crs as ccrs

# ignore all the DeprecationWarnings by pytorch
if not sys.warnoptions:
    warnings.simplefilter("ignore")

The testing device is Dell Inspirion 5680 with Intel Core i7-8700 x64 CPU and Nvidia GTX 1060 6GB GPU.<br>
Here is a benchmark about cpu v.s. gtx 1060 <br>
https://www.analyticsindiamag.com/deep-learning-tensorflow-benchmark-intel-i5-4210u-vs-geforce-nvidia-1060-6gb/

In [2]:
# constants
constant = {'g' : 9.80616,      # gravititional acceleration [m / s2]
            'R' : 6371009,      # radius of the earth [m]
            'cp': 1004.64,      # heat capacity of air [J/(Kg*K)]
            'Lv': 2500000,      # Latent heat of vaporization [J/Kg]
            'R_dry' : 286.9,    # gas constant of dry air [J/(kg*K)]
            'R_vap' : 461.5,    # gas constant for water vapour [J/(kg*K)]
            'rho' : 1026,       # sea water density [kg/m3]
            }

In [3]:
################################################################################# 
#########                           datapath                             ########
#################################################################################
# datapath for output
output_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\Lorenz84'

In [4]:
if __name__=="__main__":
    #################################################################################
    ###########                configure Lorenz 84 model                  ###########
    #################################################################################
    # Lorenz paramters and initial conditions
    x_init = 1.0 # strength of the symmetric globally encircling westerly current
    y_init = 1.0 # strength of the cosine phases of a chain of superposedwaves (large scale eddies)
    z_init = 1.0 # strength of the sine phases of a chain of superposedwaves (large scale eddies)
    F = 8.0 # thermal forcing term
    G = 1.0 # thermal forcing term
    epsilon = 0.4 # intensity of periodic forcing
    a = 0.25 # stiffness factor for westerly wind x
    b = 4.0 # advection strength of the waves by the westerly current
    
    # assuming the damping time for the waves is 5 days (Lorens 1984)
    dt = 0.0333 # 1/30 unit of time unit (5 days)
    num_steps = 1500
    # cut-off point of initialization period
    cut_off = 0
    # target testing period
    test_len = 200

In [5]:
    #################################################################################
    ###########                     Lorens 84 model                       ###########
    #################################################################################    
    def lorenz84(x, y, z, a = 0.25, b = 4.0, F = 8.0, G = 1.0):
        """
        Solver of Lorens-84 model.
        param x, y, z: location in a 3D space
        param a, b, F, G: constants and forcing
        """
        dx = - y**2 - z**2 - a * x + a * F
        dy = x * y - b * x * z - y + G
        dz = b * x * y + x * z - z
        
        return dx, dy, dz
    
    #################################################################################
    ###########            Lorens 84 model + periodic forcing             ###########
    #################################################################################
    def lorenz84_ex(x, y, z, t, a = 0.25, b = 4.0, F = 8.0, G = 1.0, epsilon = 1.0):
        """
        Solver of Lorens-84 model with periodic external forcing.
        
        param x, y, z: location in a 3D space
        param a, b, F, G: constants and forcing
        
        The model is designed with a reference to the paper:
        Broer, H., Simó, C., & Vitolo, R. (2002). Bifurcations and strange
        attractors in the Lorenz-84 climate model with seasonal forcing. Nonlinearity, 15(4), 1205.
        
        Song, Y., Yu, Y., & Wang, H. (2011, October). The stability and chaos analysis of the
        Lorenz-84 atmosphere model with seasonal forcing. In 2011 Fourth International Workshop
        on Chaos-Fractals Theories and Applications (pp. 37-41). IEEE.
        """
        # each time step is ~ 5days, therefore the returning period are 365 / 5 = 73 times in a year
        T = 73
        omega = 2 * np.pi / T
        dx = - y**2 - z**2 - a * x + a * F * (1 + epsilon * np.cos(omega * t))
        dy = x * y - b * x * z - y + G * (1 + epsilon * np.sin(omega * t))
        dz = b * x * y + x * z - z
        
        return dx, dy, dz

In [6]:
    #################################################################################
    ###########               netCDF module for saving                    ###########
    #################################################################################
    class savenc:
        def __init__(self, datapath, filename):
            """
            Save the output fields into netCDF files.
            """
            print ("Save output fields as netCDF4 files.")
            self.datapath = datapath
            # check if file exists
            if os.path.isfile(os.path.join(datapath, filename)):
                print ("File already exist.")
                raise ValueError('Should use a different name.')
            else:
                self.fileName = filename
        
        def ncfile(self, numpyMatrix):
            """
            Create nc4 file with given dataset. The dataset should contain 4 dimensions.
            """
            # check if the file exist or not
            if os.path.exists(os.path.join(self.datapath,self.fileName)):
                os.remove(os.path.join(self.datapath,self.fileName))
            data_wrap = Dataset(os.path.join(self.datapath,self.fileName), 'w',format = 'NETCDF4')
            # get the dimension of input data
            var, time = numpyMatrix.shape
            # create dimensions for netcdf data
            var_wrap_dim = data_wrap.createDimension('var',var)
            time_wrap_dim = data_wrap.createDimension('time',time)
            # create 1-dimension variables
            var_wrap_var = data_wrap.createVariable('var',np.int32,('var',))
            time_wrap_var = data_wrap.createVariable('time',np.int32,('time',))
            # create 2-dimension variables
            series_wrap_var = data_wrap.createVariable('series',np.float32,('var','time'))
            # global attributes
            data_wrap.description = 'Variables in Laurenz 84 model'
            # variable attributes
            var_wrap_var.units = '1'
            time_wrap_var.units = '1'

            var_wrap_var.long_name = 'number of variables'
            time_wrap_var.long_name = 'time span'
            # writing data
            var_wrap_var[:] = var
            time_wrap_var[:] = time
            series_wrap_var[:] = numpyMatrix
            # close the file
            data_wrap.close()
            print("Create netcdf files successfully!!")

#### Ensemble<br>
##### Testing benchmark <br>
- Lorenz 84 model initial set-up <br>
**x=1.0, y=1.0, z=1.0, a=0.25, b=4.0, F=8.0, G=1.0** <br>
**x=1.0, y=1.0, z=1.0, a=0.25, b=4.0, F=8.0, G=1.0, epsilon=0.4** <br>

##### Testing 
- Testing initial conditions x, y, z - uncertainty of model outcome <br>
**starting point x=1.0, y=1.0, z=1.0** <br>
Ensemble variant x 50 members <br>
x = [1.0020 : 0.9980] **~0.2%**<br>
x = [1.0010 : 0.9990] **~0.1%**<br>
x = [1.00010 : 0.99990] **~0.01%**<br>
x_ex = [1.0010 : 0.9990] **~0.1%**<br>
x_ex = [1.00010 : 0.99990] **~0.01%**<br>
x_ex = [1.000010 : 0.999990] **~0.001%**<br>
========================================================================================================<br>
- Testing model parameters a & b - model uncertainty <br>
**starting point a=0.25, b=4.0** <br>
Ensemble variant a <br>
a = [0.2501 : 0.24990] **~0.04%**<br>
========================================================================================================<br>
- Testing external forcing terms F & G - uncertainty related to extra/bias/trend terms <br>
**starting point F=8.0, G=1.0** <br>
Ensemble variant G <br>
G = [1.0010 : 0.9990] **~0.1%**<br>
**starting point F=8.0, G=1.0, epsilon=0.4** <br>

In [7]:
    #################################################################################
    ###########                 perturb model parameters                  ###########
    #################################################################################
    # sensitivity test parameter
    sense = 'x'
    #sense = 'a'
    #sense = 'G'
    # model type
    #model_type = 'lorenz84' # choose lorenz84 or lorenz84ex
    model_type = 'lorenz84ex'
    # ensemble members
    ens_num = 50
    # pertubation
    x_max = 1.000010
    x_min = 0.999990
    
    x_pool = np.linspace(x_min, x_max, ens_num + 1)
    
    a_max = 0.25010
    a_min = 0.24990
    
    a_pool = np.linspace(a_min, a_max, ens_num + 1)
    
    G_max = 1.0010
    G_min = 0.9990
    
    G_pool = np.linspace(G_min, G_max, ens_num + 1)

In [13]:
print(G_pool)

[0.999   0.99904 0.99908 0.99912 0.99916 0.9992  0.99924 0.99928 0.99932
 0.99936 0.9994  0.99944 0.99948 0.99952 0.99956 0.9996  0.99964 0.99968
 0.99972 0.99976 0.9998  0.99984 0.99988 0.99992 0.99996 1.      1.00004
 1.00008 1.00012 1.00016 1.0002  1.00024 1.00028 1.00032 1.00036 1.0004
 1.00044 1.00048 1.00052 1.00056 1.0006  1.00064 1.00068 1.00072 1.00076
 1.0008  1.00084 1.00088 1.00092 1.00096 1.001  ]


In [8]:
    #################################################################################
    ###########            Launch models and generate ensembles           ###########
    #################################################################################
    print("model type {}".format(model_type))
    print("sensitivity test of {}".format(sense))
    if model_type == 'lorenz84':
        for n in range(ens_num + 1):
            print("Working on ensemble member No. {}".format(n))
            #################################################################################
            ###########                 Launch Lorenz 84 model                    ###########
            #################################################################################
            # Need one more for the initial values
            x = np.empty(num_steps)
            y = np.empty(num_steps)
            z = np.empty(num_steps)

            # perturbation
            if sense == 'x':
                x[0] = x_pool[n]
            elif sense == 'a':
                a = a_pool[n]
                x[0] = x_init
            elif sense == 'G':
                G = G_pool[n]
                x[0] = x_init
            else:
                raise IOError("The chosen variable is not supported for sensitivity experiment!")
        
            # save initial values
            y[0] = y_init
            z[0] = z_init
    
            # Step through "time", calculating the partial derivatives at the current point
            # and using them to estimate the next point
            for i in range(num_steps-1):
                dx, dy, dz = lorenz84(x[i], y[i], z[i], a, b ,F, G)
                x[i + 1] = x[i] + (dx * dt)
                y[i + 1] = y[i] + (dy * dt)
                z[i + 1] = z[i] + (dz * dt)
            #################################################################################
            ###########              Save Lorenz 84 model output                  ###########
            #################################################################################
            if sense == 'x':
                ncKey = savenc(output_path, 'lorenz84_series_a{}_b{}_F{}_G{}_ens_{}.nc'.format(a, b, F, G, n))
            elif sense == 'a':
                ncKey = savenc(output_path, 'lorenz84_series_x{}_b{}_F{}_G{}_ens_{}.nc'.format(x_init, b, F, G, n))
            elif sense == 'G':
                ncKey = savenc(output_path, 'lorenz84_series_x{}_a{}_b{}_F{}_ens_{}.nc'.format(x_init, a, b, F, n))
            else:
                raise IOError("The chosen variable is not supported for sensitivity experiment!")
            nc_array = np.zeros((3,num_steps),dtype=float)
            nc_array[0,:] = x[:]
            nc_array[1,:] = y[:]
            nc_array[2,:] = z[:]
            ncKey.ncfile(nc_array)
            
    elif model_type == 'lorenz84ex':
        for n in range(ens_num + 1):
            print("Working on ensemble member No. {}".format(n))
            #################################################################################
            ###########        Launch Lorenz 84 model with periodic forcing       ###########
            #################################################################################
            # Need one more for the initial values
            x = np.empty(num_steps)
            y = np.empty(num_steps)
            z = np.empty(num_steps)
            t = 0.0

            # perturbation
            if sense == 'x':
                x[0] = x_pool[n]
            elif sense == 'a':
                a = a_pool[n]
                x[0] = x_init
            elif sense == 'G':
                G = G_pool[n]
                x[0] = x_init
            else:
                raise IOError("The chosen variable is not supported for sensitivity experiment!")
        
            # save initial values
            y[0] = y_init
            z[0] = z_init
    
            # Step through "time", calculating the partial derivatives at the current point
            # and using them to estimate the next point
            for i in range(num_steps-1):
                dx, dy, dz = lorenz84_ex(x[i], y[i], z[i], t, a, b ,F, G, epsilon)
                x[i + 1] = x[i] + (dx * dt)
                y[i + 1] = y[i] + (dy * dt)
                z[i + 1] = z[i] + (dz * dt)
                t += dt
        
            #################################################################################
            ###########              Save Lorenz 84 model output                  ###########
            #################################################################################
            if sense == 'x':
                ncKey = savenc(output_path, 'lorenz84_ex_series_a{}_b{}_F{}_G{}_ens_{}.nc'.format(a, b, F, G, n))
            elif sense == 'a':
                ncKey = savenc(output_path, 'lorenz84_ex_series_x{}_b{}_F{}_G{}_ens_{}.nc'.format(x_init, b, F, G, n))
            elif sense == 'G':
                ncKey = savenc(output_path, 'lorenz84_ex_series_x{}_a{}_b{}_F{}_ens_{}.nc'.format(x_init, a, b, F, n))
            else:
                raise IOError("The chosen variable is not supported for sensitivity experiment!")
            nc_array = np.zeros((3,num_steps),dtype=float)
            nc_array[0,:] = x[:]
            nc_array[1,:] = y[:]
            nc_array[2,:] = z[:]
            ncKey.ncfile(nc_array)
    else:
        raise IOError("The chosen variable is not supported for sensitivity experiment!")

model type lorenz84ex
sensitivity test of x
Working on ensemble member No. 0
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 1
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 2
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 3
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 4
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 5
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 6
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 7
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on ensemble member No. 8
Save output fields as netCDF4 files.
Create netcdf files successfully!!
Working on 